<a href="https://colab.research.google.com/github/adelriscom/DataScience/blob/main/ETL_with_Python_exercise_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#You have been supplied with the prices of cars in different file formats zipped in a compressed folder. 
#You have been tasked to extract all the different file formats and process them into a common/appropriate dataframes.
#You have also been tasked to conduct a simple transformation (douuble in price and round up to 2 place of decimal).
#You have also been requested to report the log information of your ETL process

In [ ]:
# import necessary libraries
import logging # for logging information
import zipfile # for unzipping files
import pandas as pd # for reading and transforming data in csv, json, and xml formats
import xml.etree.ElementTree as ET # for processing xml files
from datetime import datetime # for timestamping log messages
import glob # for finding files that match a certain pattern
import os # for manipulating file paths

In [ ]:
# define file paths and names
tmpfile = "temp.tmp" # temporary file to store extracted data
logfile = "logfile.txt" # file to store log messages
targetfile = "transformed_data.csv" # final transformed data file



In [ ]:
# download the data zip file
import subprocess
try:
    subprocess.check_call(['wget', 'https://elasticbeanstalk-us-east-2-340729127361.s3.us-east-2.amazonaws.com/prices.zip'])
except subprocess.CalledProcessError as exc:
    print(f"wget failed: {str(exc)}")
except NameError as exc:
    print(f"wget not installed: {str(exc)}")

In [ ]:
# unzip the data file
!unzip prices.zip

Archive:  prices.zip
  inflating: car_prices1.csv         
  inflating: car_prices1.json        
  inflating: car_prices1.xml         
  inflating: car_prices2.csv         
  inflating: car_prices2.json        
  inflating: car_prices2.xml         
  inflating: car_prices3.csv         
  inflating: car_prices3.json        
  inflating: car_prices3.xml         


In [51]:
# define functions to extract data from csv, json, and xml files
def extract_csv(process_file):
    dataframe = pd.read_csv(process_file)
    return dataframe

def extract_json(process_file):
    dataframe = pd.read_json(process_file, lines=True)
    return dataframe

def extract_xml(process_file):
    tree = ET.parse(process_file)
    root = tree.getroot()
    data = []
    for row in root:
        car_mode = row.find("car_mode")
        if car_mode is not None:
            car_mode = car_mode.text
        year_of_manufacture = row.find("year_of_manufacture")
        if year_of_manufacture is not None:
            year_of_manufacture = int(year_of_manufacture.text)
        price = row.find("price")
        if price is not None:
            price = float(price.text)
        data.append({'car_mode': car_mode, 'year_of_manufacture': year_of_manufacture, 'price': price})
    return pd.DataFrame(data)

In [52]:
# define a function to extract all data from csv, json, and xml files
def extract(file_path):
    # Log the start time of the extract step
    logging.info('Extract started')
    
    # Extract all the files in the zip archive to a temporary directory
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall('temp')
    
    # Create an empty DataFrame to hold extracted data
    extracted_data = pd.DataFrame(columns=['car_mode','year_of_manufacture','price', 'fuel'])
    
    # Process all csv files in the temporary directory
    for csvfile in glob.glob("temp/*.csv"):
        extracted_data = extracted_data.append(extract_csv(csvfile), ignore_index=True)
        
    # Process all json files in the temporary directory
    for jsonfile in glob.glob("temp/*.json"):
        extracted_data = extracted_data.append(extract_json(jsonfile), ignore_index=True)
        
    # Process all xml files in the temporary directory
    for xmlfile in glob.glob("temp/*.xml"):
        extracted_data = extracted_data.append(extract_xml(xmlfile), ignore_index = True)
    
    # Log the end time of the extract step
    logging.info('Extract ended')
    
    # Return the extracted data
    return extracted_data

In [53]:
# define a function to double the price in the data and round it to two decimal places
def transform(data):
    # Log the start time of the transform step
    logging.info('Transform started')
    
    # Double the price column and round to 2
    data['price'] = data['price'].apply(lambda x: round(x * 2, 2))
    
    # Log the end time of the transform step
    logging.info('Transform ended')
    
    # Return the transformed data
    return data

In [55]:
# Defining a function to load into our target file
def load(targetfile, data_to_load):
    data_to_load.to_csv(targetfile)

In [56]:
#defining the log information 

def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S'  #Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now()  #get the current timestamp
    timestamp = now.strftime(timestamp_format)
    with open ("logfile.txt", "a") as f:
        f.write(timestamp + ',' + message + '\n')

In [57]:
log ("ETL Job Started")

In [63]:
log("Extracted phase Started")
file_path = "/content/prices.zip.1"
extracted_data = extract(file_path)
log("Extracted phase Ended")
extracted_data

<ipython-input-52-f432e81e786a>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_csv(csvfile), ignore_index=True)
<ipython-input-52-f432e81e786a>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_csv(csvfile), ignore_index=True)
<ipython-input-52-f432e81e786a>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_csv(csvfile), ignore_index=True)
<ipython-input-52-f432e81e786a>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_json(jsonfile), ignore_index=

,car_mode,year_of_manufacture,price,fuel,car_model
0,NaN,2017,4253.731343,Petrol,alto 800
1,NaN,2015,10223.880597,Diesel,ciaz
2,NaN,2015,11194.029851,Petrol,ciaz
3,NaN,2015,9104.477612,Petrol,ertiga
4,NaN,2009,3358.208955,Petrol,dzire
...,...,...,...,...,...
85,None,2006,3731.343284,NaN,NaN
86,None,2010,52238.805970,NaN,NaN
87,None,2012,8805.970149,NaN,NaN
88,None,2013,5149.253731,NaN,NaN


In [64]:
log("Transform phase started")
transformed_data = transform (extracted_data)
log("Transformed phase Ended")
transformed_data

,car_mode,year_of_manufacture,price,fuel,car_model
0,NaN,2017,8507.46,Petrol,alto 800
1,NaN,2015,20447.76,Diesel,ciaz
2,NaN,2015,22388.06,Petrol,ciaz
3,NaN,2015,18208.96,Petrol,ertiga
4,NaN,2009,6716.42,Petrol,dzire
...,...,...,...,...,...
85,None,2006,7462.69,NaN,NaN
86,None,2010,104477.61,NaN,NaN
87,None,2012,17611.94,NaN,NaN
88,None,2013,10298.51,NaN,NaN


In [65]:
log("Load phase Started")
load(targetfile, transformed_data)
log("Load Phase Ended")

In [66]:
log("ETL Job Ended")